## 곱셈계층

In [1]:
class MulLayer:
    def __init__(self):
        self.x=None
        self.y=None

    def forward(self,x:float, y:float):
        self.x=x
        self.y=y
        out=self.x*self.y
        return out

    def backward(self,dout):
        dx=dout*self.y
        dy=dout*self.x
        return dx, dy                   #튜플로 나옴.


In [2]:
# MulLayer 적용

apple=100
apple_num=2
tax=1.1

node1=MulLayer()
node2=MulLayer()

#순전파
apple_price=node1.forward(apple,apple_num)
price=node2.forward(apple_price,tax)

print(price)

220.00000000000003


In [3]:
#역전파
dprice=1
dapp,dtax=node2.backward(dprice)
dapple,dapple_num=node1.backward(dapp)

print(dapple, dapple_num,dtax)

2.2 110.00000000000001 200


## 덧셈 계층

In [4]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self,x,y):
        out=x+y
        return out
    
    def backward(self,dout):
        dx=dout*1
        dy=dout*1
        return dx, dy
    

In [5]:
apple_num=2
apple=100
tangerine_num=3
tangerine=150
tax=1.1

node_app=MulLayer()
node_tang=MulLayer()
node_sum=AddLayer()
node_price=MulLayer()

#순전파
apple_price=node_app.forward(apple_num,apple)
tangerine_price=node_tang.forward(tangerine_num,tangerine)
summed_price=node_sum.forward(apple_price,tangerine_price)
price=node_price.forward(summed_price,tax)

print(price)


#역전파
dprice=1
dsummed,dtax =node_price.backward(dprice)
dapp,dtang=node_sum.backward(dsummed)
dapple_num,dapple=node_app.backward(dapp)
dtangerine_num,dtangerine=node_tang.backward(dtang)
print(dapple_num,dapple,dtangerine_num,dtangerine,dtax)

715.0000000000001
110.00000000000001 2.2 165.0 3.3000000000000003 650


In [6]:
# Relu 클래스를 만들기 전에 참고할 코드
import numpy as np

x=np.array([[1.0,-0.5],[-2.0,3.0]])
print(x)

print()

mask=(x<=0)                     # 어레이에 조건을 적용하면 불리언으로 어레이가 만들어진다.
print(mask)
out=x.copy()

print()

print(out[mask])                # 조건이 참인 것만 추출

print()

out[mask]=0                     # 조건이 참인 것들에 0을 대입입
print(out)

[[ 1.  -0.5]
 [-2.   3. ]]

[[False  True]
 [ True False]]

[-0.5 -2. ]

[[1. 0.]
 [0. 3.]]


In [7]:
class Relu:
    def __init__(self):
        self.mask=None
    
    def forward(self,x):                # x가 array로 들어온다.
        self.mask=(x<=0)                # self.mask를 boolean type으로 변경
        out=x.copy()                    # x를 out에 복사를 하고
        out[self.mask]=0                # out에 조건을 적용

        return out
    
    def backward(self,dout):            
        dout[self.mask]=0
        dx=dout

        return dx

## 5.5.2 sigmoid 계층

In [8]:
class Sigmoid:
    def __init__(self):
        self.out=None
    
    def forward(self,x):
        out=1/(1+np.exp(-x))
        self.out=out
        return out
    
    def backward(self,dout):
        dx=dout*(1-self.out)*self.out
        return dx
    

# 5.6 Affine/Softmax 계층 구현하기

## 5.6.1 Affine 계층

In [9]:
import numpy as np
class Affine:
    def __init__(self, W, b):
        self.W=W
        self.b=b
        self.x=None
        self.dW=None
        self.db=None

    def forward(self,x):
        self.x=x
        out=np.dot(x,self.W)+self.b
        return out
    
    def backward(self,dout):
        dx=np.dot(dout,self.W.T)                        
        self.dW=np.dot(self.x.T,dout)                   
        self.db=np.sum(dout, axis=0)                    # 열끼리 더해준다.

        return dx                                       # dx를 return해준다. 왜? 
                                                        # 입력 x에 대한 기울기를 이전 레이어로 전달하기 위함
                                                        # 각 레이어에서 출력에 대한 기울기를 계산한 후, 그 기울기를 입력으로 다시 전달해야 신경망이 제대로 학습


                                                        #그리고 self.dW와 self.db는 가중치와 편향에 대한 기울기로 나중에 손실함수의 최소화를 위해 필요하다.

## 5.6.3 Softmax-with-Loss 계층

마지막으로 출력층에서 사용하는 소프트맥스 함수에 대해 설명하겠다.

소프트맥스 함수는 입력값을 정규화하여 출력한다.

(정규화 (Normalization): 이후 모든 클래스에 대해 지수 값을 합산하여, 각 클래스의 지수 값을 합산한 값으로 나눕니다. 이렇게 하면 출력 값들의 총합이 1이 됩니다. 이 과정은 확률 분포를 만들기 위한 정규화입니다.)

In [10]:
#아래의 softmax함수를 위해 들고왔다.
def Softmax(a):
    c=np.max(a)
    exp_a=np.exp(a-c)
    exp_sum_a=np.sum(exp_a)
    y=exp_a/exp_sum_a
    
    return y

In [11]:
#아래의 cross_entropy_error함수를 위해 들고왔다.
def cross_entropy_error(y,t):
    if y.ndim==1:
        t=t.reshape(1,t.size)
        y=y.reshape(1,y.size)

    batch_size=y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size),t]+1e-7))/batch_size

In [12]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss=None
        self.y=None
        self.t=None
    
    def forward(self,x,t):
        self.t=t
        self.y=Softmax(x)
        self.loss=cross_entropy_error(self.y,self.t)
        return self.loss
    
    def backward(self,dout=1):
        batch_size=self.t.shape[0]
        dx=(self.y-self.t)/batch_size
        return dx

# 5.7 오차역전파법 구하기

## 5.7.1 신경망 학습의 전체 그림

## 5.7.2 오차역전파법을 적용한 신경망 구현하기

In [13]:
#TwoLayerNet을 위한 함수

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape             
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx

In [14]:
#TwoLayerNet을 위한 함수

class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

#### np.nditer

flags=['multi_index']:

* 이 플래그는 각 요소의 인덱스를 추적할 수 있게 합니다.
* 반복할 때 multi_index를 사용하면 각 요소의 다차원 인덱스를 it.multi_index 속성으로 가져올 수 있습니다.

op_flags=['readwrite']:

* 이 플래그는 배열에 대한 읽기 및 쓰기 권한을 부여합니다.
* readwrite 플래그가 지정되면 배열의 각 요소를 읽을 뿐만 아니라 수정할 수도 있습니다. 이 플래그가 없다면 해당 배열은 읽기 전용입니다.

In [15]:
#TwoLayerNet을 위한 함수

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])        #배열을 이터레이터로 사용 가능
    while not it.finished:
        idx = it.multi_index                    #현재 원소의 다차원 인덱스
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

In [16]:
# TwoLayerNet
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)                                        #인덱스를 반환
        if t.ndim != 1 : t = np.argmax(t, axis=1)                       #인덱스를 반환
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):                                 #수치적 미분을 이용
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):                                           #back propagation을 이용.
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())                             #forward한 결과들이 리스트에 저장된다. Affine1->Relu1->Affine2 이런 순으로 저장된다.
        layers.reverse()                                                #Affine2->Relu1->Affine1 이렇게 리스트가 바뀐다.
        for layer in layers:
            dout = layer.backward(dout)                                 #초반에 Affine2에 dout이 1들어감. 
                                                                        #return값들을 dout에 저장해서 연쇄적으로 계산가능.

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db     #backward함수에 dw와 db를 저장했었는데 그걸 grads의 'W1'과' 'b1'에 저장
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db     #backward함수에 dw와 db를 저장했었는데 그걸 grads의 'W2'과' 'b2'에 저장

        return grads

오차역전파법이 더 빠르다. 앞으로 이것만을 사용하자.

근데 오차역전파법이 단점은 복잡해서 종종 실수를 하는데, 제대로 구현했는지 검증하고 싶다면 두 방식으로 구한 기울기가 일치함을 확인하는 작업을 기울기 확인(GRADIENT CHECK)라고 한다.

In [19]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]                   # 첫 세개의 데이터를 선택
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:4.2993767776615225e-10
b1:2.601498037051132e-09
W2:5.072369345495358e-09
b2:1.4031024525157366e-07


In [23]:
#train_neuralnet.py

# coding: utf-8
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)            # 일반성을 고려한 방어적 코드. 한 번의 에폭(epoch)에서 최소 1번 이상의 학습을 보장하기 위해서입니다.
                                                            # train_size / batch_size가 1보다 작을 수 있는 상황을 생각해보겠습니다. 
                                                            # 예를 들어, train_size = 10이고 batch_size = 20이면 train_size / batch_size = 0.5가 되어, 
                                                            # 한 에폭을 학습하는 데 필요한 배치의 수가 0.5가 됩니다. 
                                                            # 하지만 배치의 수는 1개 이상이어야 하므로 0.5는 의미가 없습니다.

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)                   #loss함수의 값들을 gradient에서 구했지만 저장을 안 했으니 다시 구해주고
    train_loss_list.append(loss)                            #그 값들을 저장해준다.
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.095 0.0961
0.9043666666666667 0.9074
0.9261833333333334 0.927
0.9380166666666667 0.9369
0.9457333333333333 0.945
0.9513333333333334 0.9501
0.9582833333333334 0.9528
0.9627666666666667 0.9597
0.9651333333333333 0.9599
0.96875 0.9628
0.97115 0.9641
0.9727833333333333 0.966
0.9735833333333334 0.9671
0.9753666666666667 0.9688
0.9771166666666666 0.9697
0.9788833333333333 0.9698
0.97965 0.9717
